# Shot Probability Model

Many factors affect the chances that a field goal attempt will be successful. While not exhaustive, the following list contains a few:

- Shooting ability of the shooter
- Proximity (and number) of the closest defender(s)
- Shot location
- Shot type

The following introduces the `Player` class of the [py_ball](https://github.com/basketballrelativity/py_ball) package by exploring the `shortchartdetail` and `playbyplay` endpoints of the [stats.nba.com](https://stats.nba.com).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

from py_ball import player, playbyplay

HEADERS = {'Connection': 'close',
           'Host': 'stats.nba.com',
           'Origin': 'http://stats.nba.com',
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_2)' + \
                         'AppleWebKit/537.36 (KHTML, like Gecko) ' + \
                         'Chrome/66.0.3359.117 Safari/537.36'}

The `shotchartdetail` endpoint accepts a variety of parameters, but only a few are relevant for shot probability purposes:
- league_id: '00' for the NBA, '10' for the WNBA
- player_id: The unique identifier for the player (0 returns shot data for the whole game)
- game_id: The unique identifier for the game of interest
- season: YYYY-ZZ season year format (This only affects the league averages returned)

In [2]:
league_id = '00' #NBA
player_id = '0' #All players
game_id = '0021700608'
season = '2017-18'

shots = player.Player(headers=HEADERS,
                      endpoint='shotchartdetail',
                      league_id=league_id,
                      player_id=player_id,
                      game_id=game_id,
                      season=season)
shot_df = pd.DataFrame(shots.data['Shot_Chart_Detail'])
shot_df.head(10)

,ACTION_TYPE,EVENT_TYPE,GAME_DATE,GAME_EVENT_ID,GAME_ID,GRID_TYPE,HTM,LOC_X,LOC_Y,MINUTES_REMAINING,...,SHOT_ATTEMPTED_FLAG,SHOT_DISTANCE,SHOT_MADE_FLAG,SHOT_TYPE,SHOT_ZONE_AREA,SHOT_ZONE_BASIC,SHOT_ZONE_RANGE,TEAM_ID,TEAM_NAME,VTM
0,Jump Shot,Made Shot,20180110,7,0021700608,Shot Chart Detail,HOU,226,151,11,...,1,27,1,3PT Field Goal,Right Side Center(RC),Above the Break 3,24+ ft.,1610612745,Houston Rockets,POR
1,Pullup Jump shot,Missed Shot,20180110,9,0021700608,Shot Chart Detail,HOU,123,105,11,...,1,16,0,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612757,Portland Trail Blazers,POR
2,Jump Shot,Missed Shot,20180110,11,0021700608,Shot Chart Detail,HOU,-162,187,11,...,1,24,0,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612745,Houston Rockets,POR
3,Jump Shot,Missed Shot,20180110,13,0021700608,Shot Chart Detail,HOU,144,253,10,...,1,29,0,3PT Field Goal,Right Side Center(RC),Above the Break 3,24+ ft.,1610612757,Portland Trail Blazers,POR
4,Driving Layup Shot,Missed Shot,20180110,15,0021700608,Shot Chart Detail,HOU,-29,6,10,...,1,2,0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612745,Houston Rockets,POR
5,Pullup Jump shot,Missed Shot,20180110,18,0021700608,Shot Chart Detail,HOU,-214,126,10,...,1,24,0,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612745,Houston Rockets,POR
6,Jump Shot,Missed Shot,20180110,20,0021700608,Shot Chart Detail,HOU,-242,62,10,...,1,24,0,3PT Field Goal,Left Side(L),Left Corner 3,24+ ft.,1610612757,Portland Trail Blazers,POR
7,Turnaround Fadeaway shot,Made Shot,20180110,22,0021700608,Shot Chart Detail,HOU,57,88,10,...,1,10,1,2PT Field Goal,Right Side(R),In The Paint (Non-RA),8-16 ft.,1610612745,Houston Rockets,POR
8,Layup Shot,Missed Shot,20180110,23,0021700608,Shot Chart Detail,HOU,-2,62,9,...,1,6,0,2PT Field Goal,Center(C),In The Paint (Non-RA),Less Than 8 ft.,1610612757,Portland Trail Blazers,POR
9,Driving Finger Roll Layup Shot,Missed Shot,20180110,25,0021700608,Shot Chart Detail,HOU,-11,31,9,...,1,3,0,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612745,Houston Rockets,POR


In [3]:
list(shot_df)

['ACTION_TYPE',
 'EVENT_TYPE',
 'GAME_DATE',
 'GAME_EVENT_ID',
 'GAME_ID',
 'GRID_TYPE',
 'HTM',
 'LOC_X',
 'LOC_Y',
 'MINUTES_REMAINING',
 'PERIOD',
 'PLAYER_ID',
 'PLAYER_NAME',
 'SECONDS_REMAINING',
 'SHOT_ATTEMPTED_FLAG',
 'SHOT_DISTANCE',
 'SHOT_MADE_FLAG',
 'SHOT_TYPE',
 'SHOT_ZONE_AREA',
 'SHOT_ZONE_BASIC',
 'SHOT_ZONE_RANGE',
 'TEAM_ID',
 'TEAM_NAME',
 'VTM']

Looks like the `shotchartdetail` endpoint has shot location (including zone and coordinates) and type. However, defender and shot clock information do not seem to be available. To calculate shooter ability, the `playerdashboardbyshootingsplits` endpoint may be of use.

In [4]:
shooters = player.Player(headers=HEADERS,
                         endpoint='playerdashboardbyshootingsplits',
                         league_id=league_id,
                         player_id='2772',
                         season=season)
shot_type_df = pd.DataFrame(shooters.data['ShotTypePlayerDashboard'])
shot_type_df.head(10)

,BLKA,BLKA_RANK,CFID,CFPARAMS,EFG_PCT,EFG_PCT_RANK,FG3A,FG3A_RANK,FG3M,FG3M_RANK,...,PCT_AST_3PM,PCT_AST_3PM_RANK,PCT_AST_FGM,PCT_AST_FGM_RANK,PCT_UAST_2PM,PCT_UAST_2PM_RANK,PCT_UAST_3PM,PCT_UAST_3PM_RANK,PCT_UAST_FGM,PCT_UAST_FGM_RANK
0,0,1,50,Alley Oop Dunk Shot,0.000,24,0,8,0,8,...,0.0,8,0.000,21,0.000,15,0.0,5,0.000,18
1,0,1,50,Alley Oop Layup shot,0.000,24,0,8,0,8,...,0.0,8,0.000,21,0.000,15,0.0,5,0.000,18
2,0,1,50,Cutting Dunk Shot,1.000,3,0,8,0,8,...,0.0,8,1.000,1,0.000,15,0.0,5,0.000,18
3,0,1,50,Cutting Finger Roll Layup Shot,0.000,24,0,8,0,8,...,0.0,8,0.000,21,0.000,15,0.0,5,0.000,18
4,2,40,50,Cutting Layup Shot,0.667,16,0,8,0,8,...,0.0,8,1.000,1,0.000,15,0.0,5,0.000,18
5,0,1,50,Driving Bank Hook Shot,0.000,24,0,8,0,8,...,0.0,8,0.000,21,0.000,15,0.0,5,0.000,18
6,0,1,50,Driving Dunk Shot,1.000,3,0,8,0,8,...,0.0,8,0.833,8,0.167,14,0.0,5,0.167,16
7,0,1,50,Driving Finger Roll Layup Shot,0.909,10,0,8,0,8,...,0.0,8,0.800,9,0.200,13,0.0,5,0.200,15
8,0,1,50,Driving Floating Bank Jump Shot,0.750,13,0,8,0,8,...,0.0,8,0.667,11,0.333,11,0.0,5,0.333,11
9,3,43,50,Driving Floating Jump Shot,0.250,23,0,8,0,8,...,0.0,8,0.500,15,0.500,9,0.0,5,0.500,7


In [5]:
list(shot_type_df)

['BLKA',
 'BLKA_RANK',
 'CFID',
 'CFPARAMS',
 'EFG_PCT',
 'EFG_PCT_RANK',
 'FG3A',
 'FG3A_RANK',
 'FG3M',
 'FG3M_RANK',
 'FG3_PCT',
 'FG3_PCT_RANK',
 'FGA',
 'FGA_RANK',
 'FGM',
 'FGM_RANK',
 'FG_PCT',
 'FG_PCT_RANK',
 'GROUP_SET',
 'GROUP_VALUE',
 'PCT_AST_2PM',
 'PCT_AST_2PM_RANK',
 'PCT_AST_3PM',
 'PCT_AST_3PM_RANK',
 'PCT_AST_FGM',
 'PCT_AST_FGM_RANK',
 'PCT_UAST_2PM',
 'PCT_UAST_2PM_RANK',
 'PCT_UAST_3PM',
 'PCT_UAST_3PM_RANK',
 'PCT_UAST_FGM',
 'PCT_UAST_FGM_RANK']

The `shot_type_df` has a `CFPARAMS` field which seems to correspond to `ACTION_TYPE` in `shot_df`.

In [6]:
list(set(shot_df['ACTION_TYPE']))

['Turnaround Fadeaway shot',
 'Driving Finger Roll Layup Shot',
 'Putback Layup Shot',
 'Driving Layup Shot',
 'Dunk Shot',
 'Jump Shot',
 'Alley Oop Dunk Shot',
 'Driving Floating Jump Shot',
 'Running Reverse Layup Shot',
 'Layup Shot',
 'Fadeaway Jump Shot',
 'Pullup Jump shot',
 'Tip Layup Shot',
 'Step Back Jump shot',
 'Reverse Layup Shot',
 'Driving Dunk Shot',
 'Putback Dunk Shot',
 'Cutting Dunk Shot',
 'Floating Jump shot',
 'Driving Reverse Layup Shot',
 'Turnaround Jump Shot',
 'Cutting Layup Shot',
 'Running Layup Shot',
 'Turnaround Fadeaway Bank Jump Shot',
 'Hook Shot']

In [8]:
list(set(shot_type_df['CFPARAMS']))

['Driving Finger Roll Layup Shot',
 'Putback Layup Shot',
 'Turnaround Fadeaway shot',
 'Alley Oop Layup shot',
 'Alley Oop Dunk Shot',
 'Driving Layup Shot',
 'Dunk Shot',
 'Hook Bank Shot',
 'Jump Shot',
 'Running Alley Oop Layup Shot',
 'Jump Bank Shot',
 'Driving Floating Bank Jump Shot',
 'Driving Bank Hook Shot',
 'Reverse Dunk Shot',
 'Driving Floating Jump Shot',
 'Running Alley Oop Dunk Shot',
 'Running Reverse Layup Shot',
 'Running Jump Shot',
 'Layup Shot',
 'Running Finger Roll Layup Shot',
 'Fadeaway Jump Shot',
 'Pullup Jump shot',
 'Tip Layup Shot',
 'Reverse Layup Shot',
 'Step Back Jump shot',
 'Driving Dunk Shot',
 'Tip Dunk Shot',
 'Finger Roll Layup Shot',
 'Cutting Finger Roll Layup Shot',
 'Running Pull-Up Jump Shot',
 'Driving Hook Shot',
 'Turnaround Bank Hook Shot',
 'Putback Dunk Shot',
 'Cutting Dunk Shot',
 'Floating Jump shot',
 'Driving Reverse Layup Shot',
 'Turnaround Jump Shot',
 'Cutting Layup Shot',
 'Running Layup Shot',
 'Step Back Bank Jump Shot',

In [9]:
shot_area_df = pd.DataFrame(shooters.data['ShotAreaPlayerDashboard'])
shot_area_df.head(10)

,BLKA,BLKA_RANK,CFID,CFPARAMS,EFG_PCT,EFG_PCT_RANK,FG3A,FG3A_RANK,FG3M,FG3M_RANK,...,PCT_AST_3PM,PCT_AST_3PM_RANK,PCT_AST_FGM,PCT_AST_FGM_RANK,PCT_UAST_2PM,PCT_UAST_2PM_RANK,PCT_UAST_3PM,PCT_UAST_3PM_RANK,PCT_UAST_FGM,PCT_UAST_FGM_RANK
0,12,7,49,Restricted Area,0.608,3,0,6,0,4,...,0.00,4,0.602,4,0.398,3,0.00,2,0.398,3
1,8,6,49,In The Paint (Non-RA),0.148,5,0,6,0,4,...,0.00,4,0.500,5,0.500,2,0.00,2,0.500,2
2,0,1,49,Mid-Range,0.111,6,1,5,0,4,...,0.00,4,0.000,6,1.000,1,0.00,2,1.000,1
3,0,1,49,Left Corner 3,0.640,2,75,3,32,3,...,1.00,1,1.000,1,0.000,4,0.00,2,0.000,5
4,0,1,49,Right Corner 3,0.650,1,113,2,49,2,...,1.00,1,1.000,1,0.000,4,0.00,2,0.000,5
5,1,5,49,Above the Break 3,0.491,4,271,1,89,1,...,0.91,3,0.910,3,0.000,4,0.09,1,0.090,4
6,0,1,49,Backcourt,0.000,7,2,4,0,4,...,0.00,4,0.000,6,0.000,4,0.00,2,0.000,5
